Prometheus
----------

![](https://prometheus.io/assets/architecture.png)

Quelle: [Prometheus](https://prometheus.io) 

- - -

[Prometheus](https://prometheus.io) ist ein Open-Source-Toolkit zur Systemüberwachung und -Alamierung, das ursprünglich bei SoundCloud entwickelt wurde.

Prometheus trat der [Cloud Native Computing Foundation](https://www.cncf.io/) im Jahr 2016 als zweites gehostetes Projekt nach Kubernetes bei.

Zusammen mit zwei weiteren Produkten ist der Monitoring Open Source Stack wie folgt:

* **Prometheus** ist ein Zeitreihendatenbank- (time series database) und Überwachungstool, mit dem Metrik Endpunkte abgefragt und die von diesen Endpunkten bereitgestellten Daten abgeholt und verarbeitet werden. Daten können mit PromQL abgefragt werden.
* **Grafana** ist ein Datenvisualisierungs- und Analysetool, mit dem Sie Dashboards und Diagramme für Ihre Metrikdaten erstellen können.
* **Alertmanager**, der normalerweise zusammen mit Prometheus bereitgestellt wird, bildet die Warnschicht des Stapels. Er verarbeitet von Prometheus generierte Warnungen und dedupliziert (eliminieren von redundanten Daten), gruppiert und leitet sie z.B. via E-Mail weiter.


***
### Installation

Prometheus ist am einfachsten mittels [helm](https://helm.io) installierbar.


In [ ]:
%%bash
helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
helm upgrade --install monitoring prometheus-community/kube-prometheus-stack -n monitoring --namespace monitoring --create-namespace

Die Ports der User Interfaces

* Grafana (Service: prometheus-operator-grafana) 
* Prometheus (Service: prometheus-operator-prometheus)
* Alertmanager (Service: prometheus-operator-alertmanager)

sind nur innerhalb des Cluster zugänglich. Deshalb müssen wir diese zuerst freischalten bzw. gegen Aussen öffnen:

In [ ]:
! kubectl -n monitoring patch service monitoring-grafana                      -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-prometheus   -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-alertmanager -p '{"spec": {"type": "LoadBalancer"}}'

Sind Sie freigeschaltet, können wir Sie wie andere Services via Cluster-IP und Port erreichen:

In [ ]:
%%bash
echo "Grafana UI     : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-grafana -o=jsonpath='{ .spec.ports[0].nodePort }')
echo "- User         : admin"
echo "- Password     : $(kubectl --namespace monitoring get secrets monitoring-grafana -o jsonpath="{.data.admin-password}" | base64 -d ; echo)"
echo "Prometheus UI  : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')
echo "Alertmanager UI: " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-alertmanager -o=jsonpath='{ .spec.ports[0].nodePort }')

**Grafana UI** Auf der linken Seite kann zwischen einer Reihe von vorbereitenden Dashboards ausgewählt werden, z.B. Nodes.

In der **Prometheus** Oberfläche kann mittels der Abfragesprache [PromQL](https://prometheus.io/docs/prometheus/latest/querying/basics/) gezielt Ressourcen ausgewählt werden, z.B. durch Query von `apiserver_storage_objects`.

Der **Alertmanager** dient zum Verarbeiten von Ausnahmen, z.B. 


- - -
### Custom Ressourcen

Der Prometheus [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md) stellt eine Reihe von [benutzerdefinierten Ressourcen (CRDs)](https://kubernetes.io/docs/tasks/access-kubernetes-api/extend-api-custom-resource-definitions/) für das Monitoring bereit.

* **Prometheus**, für eine Prometheus Instanz.
* **Alertmanager**, für eine Alertmanager Instanz
* **ServiceMonitor**, für die Überwachung von Kubernetes-Diensten.
* **PodMonitor**, für die Überwachung von Pods.


In [ ]:
! kubectl api-resources | grep monitoring
! echo "---------------------------------"
! kubectl explain servicemonitors                   

Services überwachen
----------

Damit eigene (Micro)Services überwacht werden können, müssen diese Metrics Informationen liefern.

Dazu müssen die (Micro)Services Metrik Informationen liefern, wie der [Auto Shop GmbH](https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/-/tree/v3.0.0).

In [ ]:
%%bash
kubectl create namespace ms-rest
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/catalog-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/customer-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/order-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/webshop-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/catalog-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/customer-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/order-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/webshop-service.yaml

echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

Für die nächsten Befehle müssen alle Container gestartet sein:

In [ ]:
! kubectl get all,servicemonitors --namespace ms-rest -l tier=microservice

Zu Testzwecken zeigen wir die Metrik Daten des Order Microserivces an.

Damit mehr Daten angezeigt werden (z.B. die Anzahl HTTP Requests), vorher das UI des Webshop bzw. von Order anwählen.


In [ ]:
%%bash
curl -s "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop/order/metrics

- - -
### Prometheus 

Die erstellten Microservices können wir nun mit Prometheus überwachen.

Dazu brauchen wir die entsprechenden Rechte:   

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: microservice
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: microservice
rules:
- apiGroups: [""]
  resources:
  - nodes
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources:
  - configmaps
  verbs: ["get"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: microservice
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: microservice
subjects:
- kind: ServiceAccount
  name: microservice
  namespace: ms-kubernetes
%EOF%


Und eine eigene Prometheus Instanz.

Im User Interface können wir dann die überwachten Targets und deren Status anschauen.

Dazu ist als Query `up` einzugeben oder via Pulldown -> Status -> Targets die überwachten Microservices anzuzeigen.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: Service
metadata:
  name: prometheus
spec:
  type: NodePort
  ports:
  - name: web
    port: 9090
    protocol: TCP
    targetPort: web
  selector:
    prometheus: prometheus
  type: LoadBalancer
---
apiVersion: monitoring.coreos.com/v1
kind: Prometheus
metadata:
  name: prometheus
spec:
  serviceAccountName: microservice
  serviceMonitorSelector:
    matchLabels:
      tier: microservice
  resources:
    requests:
      memory: 400Mi
  enableAdminAPI: true
%EOF%

echo "http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"

- - -
### Prometheus 

Die erstellten Microservices können wir nun mit einer eigenen Prometheus Instanz überwachen.

Dazu brauchen wir die entsprechenden Rechte:   

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: microservice
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: microservice
rules:
- apiGroups: [""]
  resources:
  - nodes
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources:
  - configmaps
  verbs: ["get"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: microservice
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: microservice
subjects:
- kind: ServiceAccount
  name: microservice
  namespace: ms-rest
%EOF%

und eine eigene Prometheus Instanz.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: Service
metadata:
  name: prometheus
spec:
  type: NodePort
  ports:
  - name: web
    port: 9090
    protocol: TCP
    targetPort: web
  selector:
    prometheus: prometheus
  type: NodePort
---
apiVersion: monitoring.coreos.com/v1
kind: Prometheus
metadata:
  name: prometheus
spec:
  serviceAccountName: microservice
  serviceMonitorSelector:
    matchLabels:
      tier: microservice
  ruleSelector: {}  
  ruleNamespaceSelector:
    matchLabels:
      kubernetes.io/metadata.name: ms-rest
  resources:
    requests:
      memory: 400Mi
  enableAdminAPI: true
  alerting:
    alertmanagers:
      - namespace: ms-rest
        name: alertmanager
        port: web
%EOF%

### ServiceMonitor

Zum Schluss, starten wir das Monitoring mittels `ServiceMonitor`.

Welche Microservices überwacht werden sollen bestimmt der `selector` und welche Ports `endpoints` und `port:`.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: ServiceMonitor
metadata:
  name: microservice
  labels:
    tier: microservice
spec:
  selector:
    matchLabels:
      tier: microservice
  endpoints:
  - port: web
    path: /metrics 
    interval: 30s  
%EOF%

---
### Abfragen von Werten im Prometheus UI

Im User Interface können wir die überwachten Targets und deren Status anschauen.

Dazu ist als Query `up` einzugeben oder via Pulldown -> Status -> Targets die überwachten Microservices anzuzeigen.

Ausserdem können folgende Werte ausgewertet werden:

* http_requests_total{method="GET"} - liefert Anzahl HTTP GET Aufrufe
* process_memory_usage_bytes - den Speicherverbrauch pro Pod

In [ ]:
%%bash
echo "Prometheus UI: http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"

---
## Alert Manager

Zur Prometheus Instanz erstellen wir eine Alert Manager Instanz

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: Alertmanager
metadata:
  name: alertmanager
spec:
  replicas: 1
  resources:
    requests:
      memory: 200Mi
---
apiVersion: v1
kind: Service
metadata:
  name: alertmanager
spec:
  ports:
    - name: web
      port: 9093
      targetPort: 9093
  type: NodePort      
  selector:
    alertmanager: alertmanager        
%EOF%
echo "Alert Manager UI: http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest alertmanager -o=jsonpath='{ .spec.ports[0].nodePort }')"

### Alert Rules 

Damit der Alert Manager etwas zu tun bekommt, übergeben wir diesem ein paar Aufgaben:

**Speicherverbrauch pro Pod zu hoch**

Diese Regel löst aus, wenn ein einzelner Pod mehr als 500 MB RAM verwendet.

Was macht diese Regel?
* Prüft, ob ein Pod mehr als 500 MB RAM nutzt (process_memory_usage_bytes > 500 * 1024 * 1024).
* Löst aus, wenn das 2 Minuten lang anhält.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: PrometheusRule
metadata:
  name: memory-overflow
  namespace: ms-rest
spec:
  groups:
    - name: memory.rules
      rules:
        - alert: HighMemoryUsagePerPod
          expr: process_memory_usage_bytes > 5 * 1024 * 1024
          for: 2m
          labels:
            severity: critical
          annotations:
            summary: "Hoher Speicherverbrauch erkannt"
            description: "Ein Pod verbraucht mehr als 500 MB RAM."
%EOF%

Ändert die 500 auf 5 in der memory.rules und beobachtet die Regeln in Prometheus und dem Alert-Manager

---

### Aufräumen

In [ ]:
%%bash
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/catalog-deployment.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/customer-deployment.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/order-deployment.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/webshop-deployment.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/catalog-service.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/customer-service.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/order-service.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/webshop-service.yaml
kubectl delete ns ms-rest